In [38]:
# Connetced yourself Cloud Storage
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

import os
os.chdir("/content/gdrive/MyDrive/112-2_LATIA")
os.listdir()

Mounted at /content/gdrive/


['112_student.csv',
 'edu_sample_data.csv',
 '112_students_tf.csv',
 'products.csv',
 '111_ab111_Country.csv',
 '111_ab111_School.csv',
 'TaipeiSansTCBeta-Regular.ttf',
 'Ch1-5.zip (Unzipped Files)',
 'test2.txt',
 '111_ab111_Country.gsheet',
 'chromedriver.exe',
 'LICENSE.chromedriver',
 'VideoFormat.csv',
 'requests_ptt.gsheet',
 'requests_ptt.csv',
 'selenium_ptt.csv ',
 'selenium_ptt.csv']

In [39]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [40]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500

Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.08M05OI41f/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.MQnYk7drEr/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.hPNuzHdDFt/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
gpg: cannot open '/dev/tty': No such device or address
gpg: [stdout]: write error: Broken pipe
gpg: filter_flush failed on c

In [42]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Hit:1 http://deb.debian.org/debian buster InRelease
Hit:2 http://deb.debian.org/debian buster-updates InRelease
Hit:3 http://deb.debian.org/debian-security buster/updates InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:9 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency t

In [43]:
!pip install --upgrade webdriver_manager

In [46]:
#載入需要的資源
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.headless = True
driver = webdriver.Chrome(options=chrome_options)

# 設置隱式等待時間為10秒。隱式等待是在查找元素時的一種等待方式，
# 如果WebDriver沒有找到元素，它會等待一定的時間再進行查找，直到超過設定的時間為止。
driver.implicitly_wait(10)

# 打開目標網頁
driver.get("https://www.ptt.cc/bbs/basketballTW/index.html")

# 等待網頁加載，確保所需元素已經存在於網頁中
wait = WebDriverWait(driver, 10)
articles = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "r-ent")))

# 獲取網頁源碼
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')

# 解析網頁並抓取資料
data = []
index = 1

for article in soup.select('r-ent'):
    try:
        # 解析文章資訊
        title = article.select_one(".title a").text
        author = article.select_one(".meta .author").text
        date = article.select_one(".meta .date").text
        article_url = article.select_one("a")['href']
        # 將解析的資料存儲到字典中，然後添加到列表
        temp_data = {
            "Title": title,
            "Author" : author,
            "Date": date
            #"ArticleURL" : article_url
        }
        data.append(temp_data)

        index += 1
    except:
        continue

# 將抓取的資料轉換為 DataFrame，並保存為 CSV 檔案
df = pd.DataFrame(data)
df.to_csv("/content/drive/MyDrive/112-2_LATIA/selenium_ptt.csv", encoding='utf_8_sig', index=False)

# 關閉瀏覽器
driver.quit()

print(data)

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 90
Current browser version is 124.0.6367.91 with binary path /root/.cache/selenium/chrome/linux64/124.0.6367.91/chrome
Stacktrace:
#0 0x5805432b77f9 <unknown>
#1 0x5805432573b3 <unknown>
#2 0x580542f9f016 <unknown>
#3 0x580542fc5e4a <unknown>
#4 0x580542fc199a <unknown>
#5 0x580542fbe89a <unknown>
#6 0x580542ffc00a <unknown>
#7 0x580542ff6c93 <unknown>
#8 0x580542fc8ce4 <unknown>
#9 0x580542fca4d2 <unknown>
#10 0x580543283542 <unknown>
#11 0x580543292ce7 <unknown>
#12 0x5805432929e4 <unknown>
#13 0x58054329713a <unknown>
#14 0x5805432935b9 <unknown>
#15 0x580543278e00 <unknown>
#16 0x5805432aa5d2 <unknown>
#17 0x5805432aa778 <unknown>
#18 0x5805432c2a1f <unknown>
#19 0x7e205be3fac3 <unknown>
#20 0x7e205bed1850 <unknown>
